# Big Market Sales : Data Exploration
*The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and find out the sales of each product at a particular store. This is a regression problem.*

[Data Source](https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/)

------------------

## 0. Environment Setup
Initialize the pre-required modules.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

## 1. Data Exploration
### Quick Overview
There are 2 sets of data, training and testing. It is better to combine these together, and seperate them later, to reduce the hassle when we are to clean them up later.

In [2]:
# load CSVs
train_data = pd.read_csv('../data/raw/train.csv')
print('Training data shape\t: {}'.format(train_data.shape))
test_data = pd.read_csv('../data/raw/test.csv')
print('Test data shape\t\t: {}'.format(test_data.shape))

# set source labels to ease splitting
train_data['source'] = 'train'
test_data['source'] = 'test'

# join both DataFrames
data = pd.concat([train_data, test_data], ignore_index=True, sort=False)
print('Combined data shape\t: {}'.format(data.shape))

Training data shape	: (8523, 12)
Test data shape		: (5681, 11)
Combined data shape	: (14204, 13)


The variables are described as follows:

|Variable|Description|
|--------|-----------|
|Item_Identifier|Unique product ID|
|Item_Weight|Weight of product|
|Item_Fat_Content|Whether the product is low fat or not|
|Item_Visibility|The % of total display area of all products in a store allocated to the particular product|
|Item_Type|The category to which the product belongs|
|Item_MRP|Maximum Retail Price (list price) of the product|
|Outlet_Identifier|Unique store ID|
|Outlet_Establishment_Year|The year in which store was established|
|Outlet_Size|The size of the store in terms of ground area covered|
|Outlet_Location_Type|The type of city in which the store is located|
|Outlet_Type|Whether the outlet is just a grocery store or some sort of supermarket|
|Item_Outlet_Sales|Sales of the product in the particulat store. This is the outcome variable to be predicted.|

In [3]:
# preview data
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,train
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,train
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,train
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986,train
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350,train


In [4]:
# describe numeric fields
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [5]:
# unique entried for each fields
data.apply(lambda x : len(x.unique()))

Item_Identifier               1559
Item_Weight                    416
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      4
Outlet_Location_Type             3
Outlet_Type                      4
Item_Outlet_Sales             3494
source                           2
dtype: int64

In [6]:
# get the categorical fields, excluding unneccesary ones
categorical_col = [x for x in data.dtypes.index if data.dtypes[x] == 'object' and x not in ['Item_Identifier', 'Outlet_Identifier', 'source']]

# print frequency of categories in each fields
for col in categorical_col:
    print('\nAmount of categories in {}:'.format(col))
    print(data[col].value_counts())


Amount of categories in Item_Fat_Content:
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Amount of categories in Item_Type:
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Amount of categories in Outlet_Size:
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

Amount of categories in Outlet_Location_Type:
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Amount of categories in Outlet_Type:
Supermar

## 2. Data Cleaning
### Imputation
Let's now check and deal with missing values in every fields. Keep in mind that we will ignore missing values in `Item_Outlet_Sales` as the test data combined has this field blank.

In [7]:
print('Amount of missing values in each columns:')
data.apply(lambda x : sum(x.isnull()), axis=0)

Amount of missing values in each columns:


Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

Let's fill the missing values for the numerical field, this case, just `Item_Weight`.

In [8]:
# get mean per item
item_weight_avg = data.pivot_table(values='Item_Weight', index='Item_Identifier', aggfunc=np.mean)
print('Mean for first 5 items:')
print(item_weight_avg.head())

# get bool array for missing values
missing_bool = data['Item_Weight'].isnull()

# impute data, print before and after to confirm
print('\nBefore impute\t: {} missing'.format(sum(missing_bool)))
data.loc[missing_bool, 'Item_Weight'] = data.loc[missing_bool, 'Item_Identifier'].apply(lambda x : item_weight_avg.loc[x])
print('After impute\t: {} missing'.format(sum(data['Item_Weight'].isnull())))

Mean for first 5 items:
                 Item_Weight
Item_Identifier             
DRA12                 11.600
DRA24                 19.350
DRA59                  8.270
DRB01                  7.390
DRB13                  6.115

Before impute	: 2439 missing
After impute	: 0 missing


Let's now fill the missing values for the categorical field, this case, just `Outlet_Size`.

In [9]:
# get mode per item
outlet_size_mode = data.pivot_table(values='Outlet_Size', index='Outlet_Type', aggfunc=(lambda x : x.mode()[0]))
print('Mode for first 5 items:')
print(outlet_size_mode.head())

# get bool array for missing values
missing_bool = data['Outlet_Size'].isnull()

# impute data, print before and after to confirm
print('\nBefore impute\t: {} missing'.format(sum(missing_bool)))
data.loc[missing_bool, 'Outlet_Size'] = data.loc[missing_bool, 'Outlet_Type'].apply(lambda x : outlet_size_mode.loc[x])
print('After impute\t: {} missing'.format(sum(data['Outlet_Size'].isnull())))

Mode for first 5 items:
                  Outlet_Size
Outlet_Type                  
Grocery Store           Small
Supermarket Type1       Small
Supermarket Type2      Medium
Supermarket Type3      Medium

Before impute	: 4016 missing
After impute	: 0 missing


## 3. Feature Engineering
Here, we are to modify each features that will be suitable for our models to parse later, e.g. combining, grouping, normalizing features.

### Modify `Item_Visibility`
It seems to be out of sense to have a visibility of 0%, then we mmight want to replace them with the mean.

In [10]:
# get mean of Item_Visibility of each product
item_visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')
print('Item_Visibility mean for first 5 items:')
print(item_visibility_avg.head())

# get bool array for 0 values
missing_bool = (data['Item_Visibility'] == 0)

# replace values
print('\nBefore\t: {}'.format(sum(missing_bool)))
data.loc[missing_bool, 'Item_Visibility'] = data.loc[missing_bool, 'Item_Identifier'].apply(lambda x : item_visibility_avg.loc[x])
print('After\t: {}'.format(sum(data['Item_Visibility'] == 0)))

Item_Visibility mean for first 5 items:
                 Item_Visibility
Item_Identifier                 
DRA12                   0.034938
DRA24                   0.045646
DRA59                   0.133384
DRB01                   0.079736
DRB13                   0.006799

Before	: 879
After	: 0


### Create Broad Category
As we notice, the `Item_Indentifier`'s first 2 letters identifies the type of item:

|Item_Identifier|Item Class|
|---------------|----------|
|FDxxx|Food|
|DRxxx|Drink|
|NCxxx|Non-Consumable|

This information may be useful, so it is better to create a new column for this.

In [11]:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x : x[:2]).map({'FD':'Food', 'DR':'Drink', 'NC':'Non-Consumable'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drink              1317
Name: Item_Type_Combined, dtype: int64

### Generalize `Item_Fat_Content`
There are currently multiple expression of the fat contents, we have to level this out. We also want to give Non-Consumables a different classification of fat content.

In [12]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat'})
data.loc[data['Item_Type_Combined']=='Non-Consumable', 'Item_Fat_Content'] = 'Non-Edible'
data['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

### Normalizing Establishment Year
Instead of keeping the data of esablishment year, we might just keep the years that the store has been operational. Keep in mind that the data was collected in 2013.

In [13]:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

### Encoding Categorical Fields
`sklearn` models only allow numerical inputs, thus we must encode our catergorical fields, this time using *one-hot* encoding.

In [14]:
# import encoder
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

# create new Outlet field, encoded
data['Outlet'] = enc.fit_transform(data['Outlet_Identifier'])

# encode all categorical fields
categorical_col = ['Item_Fat_Content', 'Item_Type_Combined', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Outlet']
for col in categorical_col:
    data[col] = enc.fit_transform(data[col])
    
# change all encoding to one-hot
data = pd.get_dummies(data, columns=categorical_col)

In [15]:
data.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
source                        object
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Outlet_0                       uint8
O

## 4. Data Exporting
At this point, our modified dataset is ready to be exported for the modelling stage. Keep in mind that we have to split our data back to train and test files. W

In [16]:
# split train and data
train_data = data.loc[data['source'] == 'train']
test_data = data.loc[data['source'] == 'test']

# remove irrelevant columns
train_data.drop(['source'], axis=1, inplace=True)
test_data.drop(['source', 'Item_Outlet_Sales'], axis=1, inplace=True)

# export to CSV
train_data.to_csv("../data/processed/train_clean.csv", index=False)
test_data.to_csv("../data/processed/test_clean.csv", index=False)

C:\Users\danny7899\Anaconda3\envs\datasci\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
